# Assignment 4: "Search your transcripts. You will know it to be true." (Part 2)

## © Cristian Danescu-Niculescu-Mizil 2018

## CS/INFO 4300 Language and Information

### Due by midnight on Wednesday February 28th


This is an **individual** assignment.

If you use any outside sources (e.g. research papers, StackOverflow) please list your sources.

In this assignment we will explore the tradeoffs of information retrieval systems by finding newspaper quotes from "Keeping Up With The Kardashians".

**Guidelines**

All cells that contain the blocks that read `# YOUR CODE HERE` are editable and are to be completed to ensure you pass the test-cases. Make sure to write your code where indicated.

All cells that read `YOUR ANSWER HERE` are free-response cells that are editable and are to be completed.

You may use any number of notebook cells to explore the data and test out your functions, although you will only be graded on the solution itself.


You are unable to modify the read-only cells.

You should also use Markdown cells to explain your code and discuss your results when necessary.
Instructions can be found [here](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html).

All floating point values should be printed with **2 decimal places** precision. You can do so using the built-in round function.

**Grading**

For code-completion questions you will be graded on passing the public test cases we have included, as well as any hidden test cases that we have supplemented to ensure that your logic is correct.

For free-response questions you will be manually graded on the quality of your answer.

## Find the most similar messages (cosine similarity)

### A high-level overview

Our overall goal of the last part of this assignment is to build a system where we can compute the cosine similarity between queries and our datasets quickly. To accomplish queries and compute cosine similarities, we will need to represent documents as vectors. A common method of representing documents as vectors is by using "term frequency-inverse document frequency" scores. More details about this method can be found [on the course website](http://www.cs.cornell.edu/courses/cs4300/2018sp/Slides//vsm_cheatsheet.pdf). The notation here is consistent with the hand out, so if you haven't read over it -- you should!

Consider the tf-idf representation of a document and a query: $\vec{d_j}$ and $\vec{q}$, respectively. Elements of these vectors are very often zero because the term frequency of most words in most documents is zero. Stated differently, most words don't appear in most documents! Consider a query that has 5 words in it and a vocabulary that has 20K words in it -- only .025% of the elements of the vector representation of the query are nonzero! When a vector (or a matrix) has very few nonzero entries, it is called "sparse." We can take advantage of the sparsity of tf-idf document representations to compute cosine similarity quickly. We will first build some data stuctures that allow for faster querying of statistics, and then we will build a function that quickly computes cosine similarity between queries and documents.

### A starting point
We will use an **inverted index** for efficiency. This is a sparse term-centered representation that allows us to quickly find all documents that contain a given term.

### Q1 Write a function to construct the index.

As in class, the index is a key-value structure where the keys are terms and the values are lists of *postings*. In this case, like in class, we record the documents a term occurs in as well as the **count** of that term in that document.

In [1]:
import json
with open("kardashian-transcripts.json", "r") as f:
    transcripts = json.load(f)
print(len(transcripts[0]))

851


In [2]:
flat_msgs = [m for transcript in transcripts for m in transcript]

In [3]:
from collections import defaultdict
from collections import Counter
import math
import time
import numpy as np
from nltk.tokenize import TreebankWordTokenizer

In [4]:
def build_inverted_index(msgs):
    """ Builds an inverted index from the messages.
    
    Arguments
    =========
    
    msgs: list of dicts.
        Each message in this list already has a 'toks'
        field that contains the tokenized message.
    
    Returns
    =======
    
    index: dict
        For each term, the index contains a list of
        tuples (doc_id, count_of_term_in_doc):
        index[term] = [(d1, tf1), (d2, tf2), ...]
        
    Example
    =======
    
    >> test_idx = build_inverted_index([
    ...    {'toks': ['to', 'be', 'or', 'not', 'to', 'be']},
    ...    {'toks': ['do', 'be', 'do', 'be', 'do']}])
    
    >> test_idx['be']
    [(0, 2), (1, 2)]
    
    >> test_idx['not']
    [(0, 1)]
    
    """
    # YOUR CODE HERE
    index = {}
    for i in range(len(msgs)):
        for word in set(msgs[i]['toks']):
            cnt = msgs[i]['toks'].count(word)
            if word not in index.keys():
                index[word] = [(i, cnt,)]
            else:
                index[word].append((i, cnt,))
    return index
    raise NotImplementedError()

In [5]:
# This is an autograder test. Here we can test the function you just wrote above.
start_time = time.time()
inv_idx = build_inverted_index(flat_msgs)
execution_time = time.time() - start_time

assert len(inv_idx['bruce']) >= 385 and len(inv_idx['bruce']) <= 435
assert len(inv_idx['baby']) >= 250 and len(inv_idx['baby']) <= 300
assert execution_time <= 1.0


### Q2 Compute IDF *using* the inverted index

Write a function `compute_idf` that uses the inverted index to efficiently compute IDF values.

Words that occur in a very small number of documents are not useful in many cases, so we ignore them. Use a parameter `min_df`
to ignore all terms that occur in strictly fewer than `min_df=10` documents.

Similarly, words that occur in a large *fraction* of the documents don't bring any more information for some tasks. Use a parameter `max_df_ratio` to trim out such words. For example, `max_df_ratio=0.95` means ignore all words that occur in more than 95% of the documents.

As a reminder, we define the IDF statistic as...
$$ IDF(t) = \log \left(\frac{N}{1 + DF(t)} \right) $$

where N is the total number of docs and $DF(t)$ is the number of docs containing $t$. Keep in mind, there are other definitions if IDF out there, so if you go looking for resources on the internet, you might find differing (but also valid) accounts. The base of the log can be taken to be 2 (though the base doesn't really matter).

In [6]:
def compute_idf(inv_idx, n_docs, min_df=10, max_df_ratio=0.95):
    """ Compute term IDF values from the inverted index.
    
    Words that are too frequent or too infrequent get pruned.
    
    
    Arguments
    =========
    
    inv_idx: an inverted index as above
    
    n_docs: int,
        The number of documents.
        
    min_df: int,
        Minimum number of documents a term must occur in.
        Less frequent words get ignored.
    
    max_df_ratio: float,
        Maximum ratio of documents a term can occur in.
        More frequent words get ignored.
    
    Returns
    =======
    
    idf: dict
        For each term, the dict contains the idf value.
        
    """
    # YOUR CODE HERE
    idf = {}
    for term, postings in inv_idx.items():
        if len(postings) < min_df or (len(postings)+0.)/n_docs > max_df_ratio:
            continue
        else:
            idf[term] = np.log2(n_docs/(1.0+len(postings)))
    return idf
    raise NotImplementedError()

In [7]:
# This is an autograder test. Here we can test the function you just wrote above.
start_time = time.time()
idf_dict = compute_idf(inv_idx, len(flat_msgs))
execution_time = time.time() - start_time

assert idf_dict['bruce'] >= 6.0 and idf_dict['bruce'] <= 7.0
assert idf_dict['baby'] >= 6.0 and idf_dict['baby'] <= 8.0
assert execution_time <= 0.5


### Q3 Compute the norm of each document using the inverted index

Recalling our tf-idf vector representation of documents, we can compute the "norm" as the norm (length) of the vector representation of that document. More specifically, the norm of a document $j$, denoted as $\left|\left| d_j \right|\right|$, is given as follows...

$$ \left|\left| d_j \right|\right| = \sqrt{\sum_{\text{word } i} (tf_{ij} \cdot idf_i)^2} $$

In [8]:
def compute_doc_norms(index, idf, n_docs):
    """ Precompute the euclidean norm of each document.
    
    Arguments
    =========
    
    index: the inverted index as above
    
    idf: dict,
        Precomputed idf values for the terms.
    
    n_docs: int,
        The total number of documents.
    
    Returns
    =======
    
    norms: np.array, size: n_docs
        norms[i] = the norm of document i.
    """
    
    # YOUR CODE HERE
    norms = np.zeros(n_docs)
    for term, idf_i in idf.items():
        doc_id, freq = zip(*index[term])
        tf_idf = np.square(idf_i * np.array(freq))
        for k in range(len(doc_id)):
            norms[doc_id[k]] += tf_idf[k]
    norms = np.sqrt(norms)
    return norms
    raise NotImplementedError()

In [9]:
# This is an autograder test. Here we can test the function you just wrote above.
start_time = time.time()
doc_norms = compute_doc_norms(inv_idx, idf_dict, len(flat_msgs))
execution_time = time.time() - start_time

assert doc_norms[1] >= 15.5 and doc_norms[1] <= 17.5
assert doc_norms[5] >= 6.5 and doc_norms[5] <= 8.5
assert execution_time <= 1.0


### Q4 Find the most similar messages to the quotes.  Write the `index_search` function.

The goal of this section is to implement index_search, a fast implementation of cosine similarity. Next, you can test your answer by running the search using the code provided. Briefly discuss why it worked, or why it might not have worked, for each query.

The goal of index_search is to compute the cosine similarity between the query and each document in the dataset. Naively, this computation requires you to compute dot products between the query tf-idf vector $q$ and each document's tf-idf vector $d_i$.

However, you should be able to use the sparsity of the tf-idf representation and the data structures you created to your advantage. More specifically, consider the cosine similarity...

$$ cossim(\vec{q}, \vec{d_j}) = \frac{\vec{q} \cdot \vec{d_j}}{\|\vec{q}\| \cdot \|\vec{d_j}\|}$$

Specifically, focusing on the numerator...

$$ \vec{q} \cdot \vec{d_j} = \sum_{i} {q_i} * {d_i}_j $$

Here ${q_i}$ and ${d_i}_j$ are the $i$-th dimension of the vectors $q$ and ${d_j}$ respectively.
Because many ${q_i}$ and ${d_i}_j$ are zero, it is actually a bit wasteful to actually create the vectors $q$ and $d_j$ as numpy arrays; this is the method that you saw in class.

A faster approach to computing the numerator term of cosine similarity involves quickly computing the above summation using the inverted index, pre-computed idf scores, and pre-computed document norms.

A good "first step" to implementing this efficiently is to only loop over ${q}_j$ that are nonzero (i.e. ${q}_j$ such that the word $j$ appears in the query). 

**Note** Convert the query to lowercase, and use the `nltk.tokenize.TreebankWordTokenizer` to tokenize the query. The transcripts have already been tokenized this way.


**Aside:** Precomputation

In many settings, we will need to repeat the same kind of operation many times. Often, part of the input doesn't change.
Queries against the Kardashians transcript are like this: we want to run more queries (in the real world we'd want to run a lot of them every second, even) but the data we are searching doesn't change.

We could write an `index_search` function with the same signature as `verbatim_search`, taking the `query` and the `msgs` as input, and the function would look like:

    def index_search(query, msgs):
        inv_idx = build_inverted_index(msgs)
        idf = compute_idf(inv_idx, len(msgs))
        doc_norms = compute_doc_norms(inv_idx)
        # do actual search


But notice that the first three lines only depend on the messages. Imagine if we run this a million times with different queries but the same collection of documents: we'd wastefully recompute the index, the IDFs and the norms every time and discard them. It's a better idea, then, to precompute them just once, and pass them as arguments.

In [10]:
queries = [u"It's like a bunch of people running around talking about nothing.",
           u"Never say to a famous person that this possible endorsment would bring 'er to the spot light.",
           u"Your yapping is making my head ache!",
           u"I'm going to Maryland, did I tell you?"]

In [11]:
inv_idx = build_inverted_index(flat_msgs)

idf = compute_idf(inv_idx, len(flat_msgs),
                  min_df=10,
                  max_df_ratio=0.1)  # documents are very short so we can use a small value here
                                     # examine the actual DF values of common words like "the"
                                     # to set these values

inv_idx = {key: val for key, val in inv_idx.items()
           if key in idf}            # prune the terms left out by idf

doc_norms = compute_doc_norms(inv_idx, idf, len(flat_msgs))

In [12]:
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize("a more general-purpose tokenizer")

['a', 'more', 'general-purpose', 'tokenizer']

In [13]:
def index_search(query, index, idf, doc_norms):
    """ Search the collection of documents for the given query
    
    Arguments
    =========
    
    query: string,
        The query we are looking for.
    
    index: an inverted index as above
    
    idf: idf values precomputed as above
    
    doc_norms: document norms as computed above
    
    Returns
    =======
    
    results, list of tuples (score, doc_id)
        Sorted list of results such that the first element has
        the highest score, and `doc_id` points to the document
        with the highest score.
        
    """
    
    # YOUR CODE HERE
    results = np.array([0. for i in range(len(doc_norms))])
    q = tokenizer.tokenize(query.lower())
    q_tf = {term: q.count(term) for term in set(q)}
    # Compute q_norm
    q_norm = 0.
    for term, tf in q_tf.items():
        if term in idf.keys():
            q_norm += (tf*idf[term])**2
    q_norm = np.sqrt(q_norm)
    if q_norm != 0.:
        # Compute cosine similarity
        for term_i, tf_qi in q_tf.items():
            if term_i in idf.keys():
                doc_id, tf = zip(*index[term_i])
                numerator_array = tf_qi * idf[term_i]**2 * np.array(tf)
                for k in range(len(doc_id)):
                    results[doc_id[k]] += numerator_array[k]
        res = [results[i]/(q_norm * doc_norms[i]) if results[i] != 0. and doc_norms[i] != 0. else 0.\
               for i in range(len(results))]
        res = list(zip(res, range(len(res))))
        return sorted(res, key = lambda x: x[0], reverse = True)
    else:
        return list(zip(results, range(len(results))))
    raise NotImplementedError()

In [14]:
# This is an autograder test. Here we can test the function you just wrote above.
start_time = time.time()
score, _ = index_search(queries[1], inv_idx, idf, doc_norms)[0]
execution_time = time.time() - start_time

assert score >= 0.38 and score <= 0.48
assert execution_time <= 0.5


for query in queries:
    print("#" * len(query))
    print(query)
    print("#" * len(query))

    for score, msg_id in index_search(query, inv_idx, idf, doc_norms)[:10]:
        print("[{:.2f}] {}: {}\n\t({})".format(
            score,
            flat_msgs[msg_id]['speaker'],
            flat_msgs[msg_id]['text'],
            flat_msgs[msg_id]['episode_title'])) 
    print()

#################################################################
It's like a bunch of people running around talking about nothing.
#################################################################
[1.00] BRUCE: It's like a bunch of people running around talking about nothing.
	(Keeping Up With the Kardashians - Kourt's First Cover)
[0.61] KRIS: It's not a bunch of teenagers running around.
	(Keeping Up With the Kardashians - Kris ``The Cougar'' Jenner)
[0.46] ROB: I have a bunch of connections in the industry.
	(Keeping Up With the Kardashians - Must Love Dogs)
[0.46] ROB: I have a bunch of connections in the industry.
	(Keeping Up With the Kardashians - Must Love Dogs)
[0.43] KHLOE: She's, like, running.
	(Keeping Up With the Kardashians - Match Made in Hell)
[0.42] ROB: She got me a bunch of interviews.
	(Keeping Up With the Kardashians - Must Love Dogs)
[0.42] ROB: She got me a bunch of interviews.
	(Keeping Up With the Kardashians - Must Love Dogs)
[0.41] KHLOE: Not running.
	(Kee

### Briefly discuss why it worked, or why it might not have worked, for each query.

The code above worked for the first and the fourth query because the top result matches every single word in the query. For the second and third query, it might not have worked because documents that match more words have lower scores, and the results seem to be not quite relevant to our queries.

## Q5EC: Bonus question 1 for extra course credit.

### Updating precomputed values.

In many real-world applications, the collection of documents will not stay the same forever. At Internet-scale, however, it could possibly even be worth recomputing things every second, if during that second we're going to answer millions of queries.

However, there's a better way: in reality, the document set will not change radically, but incrementally.  In particular, it's most common to add or remove a bunch of new documents to the index.

Write functions `add_docs` and `remove_docs` that update the index, idf and document norms.  Think of the implications this has on how we store the IDF. Is there a better way of storing it, that minimizes the memory we need to touch when updating?

Think of adequate test cases for these functions and implement them.

**Note:** You can get up to 0.5 EC for completing this question.

In [15]:
# YOUR CODE HERE
def add_docs(new_docs, msgs, index, compute_idf, compute_doc_norms):
    """
    input:
    new_docs: list of dicts
    msgs: list of dicts
    index: dict
    compute_idf: function, compute_idf(inv_idx, n_docs, min_df=10, max_df_ratio=0.95)
    compute_doc_norms: def compute_doc_norms(index, idf, n_docs)
    return: 
    index: dict
    idf: dict
    doc_norms: numpy array
    """
    msgs.extend(new_docs)
    
    # Update inversed index
    for i in range(len(msgs)-len(new_docs), len(msgs)):
        for term in set(msgs[i]['toks']):
            cnt = msgs[i]['toks'].count(term)
            if term not in index.keys():
                index[term] = [(i, cnt,)]
            else:
                index[term].append((i, cnt,))
    # Update idf
    new_idf = compute_idf(index, len(msgs), min_df=10, max_df_ratio=0.95)
    # Update doc_norms
    new_doc_norms = compute_doc_norms(index, new_idf, len(msgs))
    return index, new_idf, new_doc_norms
    raise NotImplementedError()

In [16]:
new_inv_idx, new_idf, new_doc_norms = \
add_docs(flat_msgs[0:3], flat_msgs, inv_idx, compute_idf, compute_doc_norms)

In [17]:
assert len(flat_msgs) == 39681
assert new_doc_norms[-3] > 17.0 and new_doc_norms[-3] < 18.0
assert new_doc_norms[-2] > 16.0 and new_doc_norms[-2] < 16.5
assert new_doc_norms[-1] > 4.0 and new_doc_norms[-1] < 4.5

In [18]:
def remove_docs(rm_docs, msgs, index, compute_idf, compute_doc_norms):
    """
    input:
    rm_docs: list of doc_id to be removed
    msgs: list of dicts
    index: dict
    compute_idf: function, compute_idf(inv_idx, n_docs, min_df=10, max_df_ratio=0.95)
    compute_doc_norms: def compute_doc_norms(index, idf, n_docs)
    return: 
    index: dict
    idf: dict
    doc_norms: numpy array
    """
    # Update inversed index
    for i in rm_docs:
        for term in set(msgs[i]['toks']):
            if term in index.keys():
                doc_id, _ = zip(*index[term])
                if i in doc_id:
                    rm_idx = doc_id.index(i)
                    index[term].pop(rm_idx)
    # Delete docs
    for j in sorted(rm_docs, reverse = True):
        msgs.pop(j)
    # Update idf
    rm_idf = compute_idf(index, len(msgs), min_df=10, max_df_ratio=0.95)
    # Update doc_norms
    rm_doc_norms = compute_doc_norms(index, rm_idf, len(msgs))
    return index, rm_idf, rm_doc_norms
    raise NotImplementedError()

In [19]:
rm_inv_idx, rm_idf, rm_doc_norms = \
remove_docs([39678, 39679, 39680], flat_msgs, new_inv_idx, compute_idf, compute_doc_norms)

In [20]:
assert len(flat_msgs) == 39678
assert rm_doc_norms[0] > 17.5 and rm_doc_norms[0] < 18.0
assert rm_doc_norms[1] > 16.0 and rm_doc_norms[1] < 16.5
assert rm_doc_norms[2] > 4.4 and rm_doc_norms[2] < 4.5

## Q6EC: Bonus question 2 for extra course credit.

### Finding your own similarity metric

We've explored using cosine similarity and edit distance to find similar messages to input queries. However, there's a whole world of ways to measure the similarity between two documents. Go forth, and research!

For this question, find a new way of measuring similarity between two documents, and implement a search using your new metric. Your new way of measuring document similarity should be different enough from the two approaches we already implemented. It can be a method you devise or an existing method from somewhere else.

**Note:** The amount of EC awarded for this question will be determined based on creativity, originality, implementation, and analysis.

In [21]:
# YOUR CODE HERE
#raise NotImplementedError()